# Images deleter

In [1]:
import os
from IPython.core.display import display, HTML
import uuid

This UI displays images from a folder, in a Jupyter Notebook, and allows to delete images quickly by clicking on them.

It has only been tested on MacOS so far.

To use, run all cells then call images_deleter("your/folder/path/")

In [2]:
%%javascript
// window.executePython() allows to call python code in javascript
window.executePython = function(python) {
    return new Promise((resolve, reject) => {
        var callbacks = {
            iopub: {
                output: (data) => resolve(data.content.text.trim())
            }
        };
        Jupyter.notebook.kernel.execute(`print(${python})`, callbacks);    
    });
}

// Example:
// window.executePython("'This is ' + 'python'")
//    .then(result => console.log(result));

<IPython.core.display.Javascript object>

In [18]:
def delete_target(folder_relative_path,target):
    '''Called by images_deleter()'''
    path = os.path.join(folder_relative_path,target)
    os.remove(path)
    return path + ' deleted'

def images_deleter(folder_relative_path, perPage = 49, perRow = 7):
    '''Displays images in a folder. Click on an image to delete it
    folder_relative_path : path of format folder/folder/
    PerPage : Number of images displayed at the same time
    perRow : Number of images per row.
    '''
    files = os.listdir(folder_relative_path)
    # Unique Id to make sure the right element is selected
    galleryId = 'gallery' + str(uuid.uuid4())
    
    buttons = "<div style='display:flex;justify-content:space-between'><button onclick = 'previous()'>« PREVIOUS</button><button onclick = 'next()'>NEXT »</button></div>"
    html = '''<style>.imgItem:hover {{transform: scale(1.5)}}</style>
        <div>Click on an image to delete it</div>
        {buttons}
        <div id = '{galleryId}' style='display:flex;flex-wrap: wrap'></div>
        {buttons}'''.format(galleryId = galleryId, buttons = buttons)
    script = '''<script>var images = {files}.map(image => `
        <div style='width:${{100 / {perRow} }}%'>
            <img src='{path}${{escape(image)}}' class = 'imgItem' alt = '${{image}}'/>
        </div>`);
        
        // Display the image gallery :
        function showPage(page = 0){{
            var firstImg = page * {perPage};
            if (firstImg >= images.length){{
                return 0
            }}
            document.getElementById('{galleryId}').innerHTML = images.slice(firstImg,Math.min(firstImg + {perPage}, images.length)).join("");
            return 1
        }}
        showPage();
            
        // Delete image on click :
        document.getElementById('{galleryId}').addEventListener('click',(e) => {{
            if(e.target.classList.contains('imgItem')){{
                var pythonCode = `delete_target('{path}','${{e.target.alt}}')`;
                window.executePython(pythonCode)
                e.target.remove();
            }}
        }});
        
        // Page navigation:
        var currentPage = 0;
        function next(){{
            if (showPage(currentPage + 1)){{
                currentPage++;
            }}
        }}
        function previous(){{
            if (currentPage > 0){{
                currentPage--;
                showPage(currentPage);
            }}
        }}</script>'''.format(galleryId = galleryId, files = files, perPage = perPage, perRow = perRow, path = folder_relative_path)
    display(HTML(html + script))

In [19]:
images_deleter("downloads/photo/")